The below notebook is intended to translate transactions from Sleeper's API into a readable message that can then be passed to a messenging service. 

First we import the necessary modules:
1. requests (to make the HTTP requests to the API)
2. json (to best read/write data to files)
3. logging (to log what we do! very important to track your work!)

In [1]:
import requests, json, logging

logger = logging.getLogger(__name__)

The next piece we are setting our league's ID number. This is my league's ID number that we will use throughout this process. When using this for other league's, you can substitute yours in. 

In [2]:
league_id = '1100536352157859840'

The first function created here is to get and store the list of players that Sleeper has. We will reference this later.

In [3]:
def get_player_list():
    return requests.get("https://api.sleeper.app/v1/players/nfl").json()

The next function here is to save our list of players locally, wherever this script is run, for accessing later. 

In [4]:
def save_player_list(player_list, indent=2):
    with open('players.txt', 'w') as f:
        json.dump(player_list, f, indent=indent)

This function is to get the teams/users in the league.

In [5]:
def get_teams(leauge_id):
    return requests.get(f"https://api.sleeper.app/v1/league/{league_id}/users").json()

This function is to get the rosters in the league.

In [6]:
def get_rosters(league_id):
    return requests.get(f"https://api.sleeper.app/v1/league/{league_id}/rosters").json()

Now, we set up a few classes that allow us to do our work. These two classes give us the ability to run methods on individual results from what is returned from the HTTP request. For now, they just give us ease of use. 

In [7]:
class Team:
    def __init__(self, team_dict):
        for key, value in team_dict.items():
            setattr(self, key, value)
        
class Roster:
    def __init__(self, roster_dict):
        for key, value in roster_dict.items():
            setattr(self, key, value)

The below class is created for the league. As mentioned before, we are setting these up to run methods on the results. Since a league request returns the league itself (rather than a list of teams/rosters), we want to add those in! There are 3 methods. 
1. Add Team List - get us all the teams in the league, transforms them into classes, stores them in a list. 
2. Add Roster List - does the same as the team list. 
3. Construct Teams - Sorts through the teams and rosters to give us the key pieces of information in a nice, easy-to-navigate dictionary. The owner of the team, the name of the team, and the roster ID number for the team. 

In [8]:
class League:
    def __init__(self, league_id):
        self.league_id = league_id
        league_dict = requests.get(f"https://api.sleeper.app/v1/league/{self.league_id}").json()
        for key, value in league_dict.items():
            setattr(self, key, value)
        self._add_team_list()
        self._add_roster_list()
        self._construct_teams()
            
    def _add_team_list(self):
        self.team_list = []
        for team_dict in get_teams(self.league_id):
            team_data = Team(team_dict)
            self.team_list.append(team_data)
        return self
    
    def _add_roster_list(self):
        self.roster_list = []
        for roster_dict in get_rosters(self.league_id):
            self.roster_list.append(Roster(roster_dict))
        return self
    
    def _construct_teams(self):
        self.teams = []
        for team in self.team_list:
            try:
                self.teams.append({'owner_id' : team.user_id,
                    'team_name' : team.metadata['team_name'],
                    'roster_id' : [x.roster_id for x in self.roster_list if x.owner_id == team.user_id][0]})
            except:
                self.teams.append({'owner_id' : team.user_id,
                    'team_name' : 'Team ' + team.display_name,
                    'roster_id' : [x.roster_id for x in self.roster_list if x.owner_id == team.user_id][0]})

This function gives us the ability to get the transactions in a given week. Note that 'round' here is meant to be the week. 

In [9]:
def get_transactions(league_id, round):
    return requests.get(f"https://api.sleeper.app/v1/league/{league_id}/transactions/{round}").json()

This function is an ease of use function to see what transactions we could come across if we pull all of them. 

In [10]:
def get_all_transaction_types(transactions):
    t_list = []
    [t_list.append(x['type']) for x in transactions if x['type'] not in t_list]
    return t_list

This is the example showing what we currently have as transactions. 

In [11]:
t = get_all_transaction_types(get_transactions(league_id, 1))
print(t)

['free_agent', 'commissioner', 'trade']


Now, here we are setting up the Transaction class that will allow us to create our readable messages. There are a few things going on. 

First, we determine what rosters are in play. We already have the roster ID included in any result from an HTTP request, but that's not at all helpful in a readable message. We have to find the team name who performed the action, and this method does that. 

Second, we determine what players are in play as well. We do not need to specify what is happening to each player yet, that we do later. For now, we just need to be able to put names to ID numbers. 

Third, is our string functionality that gives us our message! 

Fourth, is a method to log the transaction locally. The method does not return self, but a boolean, so you know if the transaction has already been logged. This will be useful when we make repeated calls to the transactions endpoint of the API. 

In [12]:
class Transaction:
    def __init__(self, transaction_dict: dict, league: League):
        self.league = league
        for key, value in transaction_dict.items():
            setattr(self, key, value)
        self._determine_rosters()
        self._determine_players()
            
    def _determine_rosters(self):
        self.teams = {}
        for roster in self.roster_ids:
            self.teams[roster] = [x['team_name'] for x in self.league.teams if x['roster_id'] == roster][0]
        return self
    
    def _determine_players(self):
        self.players = {}
        self._load_player_list()
        try:
            for add in self.adds:
                self.players[add] = self.player_dict[add]['full_name']
        except:
            logger.info('No adds detected.')
        try:
            for drop in self.drops:
                self.players[drop] = self.player_dict[drop]['full_name']
        except:
            logger.info('No drops detected.')
        self._empty_player_list()
            
    def _load_player_list(self):
        with open('players.txt', 'r') as p:
            self.player_dict = json.load(p)
        return self
    
    def _empty_player_list(self):
        self.player_dict = None
        
    def __str__(self):
        return_string_list = ['Type: ' + self.type]
        for roster in self.roster_ids:
            return_string_list.append('Team: ' + str(self.teams[roster]))
            try:
                for drop in self.drops:
                    return_string_list.append('Dropped Players: ' + self.players[drop])
            except:
                logger.info('No drops detected.')
            try:
                for add in self.adds:
                    return_string_list.append('Added Players: ' + self.players[add])
            except:
                logger.info('No adds detected.')
        return '\n'.join(return_string_list)
    
    def log_transaction(self):
        with open('transactions.txt', 'r') as f:
            t = json.load(f)
            if self.transaction_id in t:
                logger.info('Transaction already recorded.')
                return False
            else:
                t.append(self.transaction_id)
                logger.info('Transaction successfully logged.')
                return True

Here, we quickly set ourselves up to see the output of our work. 

In [13]:
t = get_transactions(league_id, 1)
league = League(league_id)

This lets us view some examples of our work. 

In [14]:
fa = True
comm = True
trade = True

for key, transaction in enumerate(t):
    x = str(Transaction(transaction, league))
    if transaction['type'] == 'free_agent':
        if fa:
            print(x)
            fa = False
    elif transaction['type'] == 'commissioner':
        if comm:
            print(x)
            comm = False
    elif transaction['type'] == 'trade':
        if trade:
            print(x)
            trade = False

Type: free_agent
Team: The Adams Family
Dropped Players: Ainias Smith
Added Players: Dillon Johnson
Type: commissioner
Team: Team SleeperShane
Added Players: Kendall Milton
Type: trade
Team: The Adams Family
Team: Big Data Big Nerds
